# 🖼️ 02 - Création de masques

### Représentation des Images sous forme de Données

Une image, qu'elle soit en noir et blanc ou en couleur, peut être interprétée comme une **matrice de données**. Chaque pixel de l'image est représenté par une valeur numérique, et l'image entière peut être vue comme une grille de ces pixels disposés selon les dimensions **Largeur (Width) x Hauteur (Height)**.

#### 1. Image en Noir et Blanc
Dans une image en **noir et blanc**, chaque pixel est représenté par une **valeur unique** qui indique son niveau de gris, variant de **0 à 255**.  
- **0** représente le noir pur,  
- **255** représente le blanc pur,  
- Les valeurs intermédiaires correspondent à des nuances de gris.

L'image en noir et blanc devient ainsi une **matrice 2D**, où chaque pixel est assigné à une valeur de gris entre 0 et 255.  

**[[  0,  50, 100],  
 [150, 200, 255],  
 [128,  64,  32]]**

#### 2. Image en Couleur (Système RGB)
Les images en **couleur** sont encodées avec le système **RGB** (Rouge, Vert, Bleu). Chaque pixel de l'image est composé de trois valeurs :  
- Une pour le **Rouge (R)**,  
- Une pour le **Vert (G)**,  
- Une pour le **Bleu (B)**.

Chaque composante varie entre **0 et 255**, où **0** représente l'absence de couleur et **255** indique une intensité maximale.  
Cela donne une **matrice 3D** où chaque pixel est représenté par un triplet de valeurs (**R, G, B**) et l'image entière devient une matrice de forme **Largeur x Hauteur x 3**.

**[[[255,   0,   0], [  0, 255,   0], [  0,   0, 255]],  
 [[255, 255,   0], [  0, 255, 255], [255,   0, 255]],  
 [[128, 128, 128], [  0,   0,   0], [255, 255, 255]]]**

Il existe une exemple interactif ici qui permet de visualiser comment une image est encodée, et interpretée numériquement.
https://setosa.io/ev/image-kernels/

---

### Transition vers les Masques Binaires en Segmentation d'Images

Dans le cadre de la **segmentation d'images**, il est essentiel de créer des **masques binaires** qui serviront de **vérité terrain** pour l'entraînement du modèle. Ces masques sont similaires aux images classiques, mais avec une différence clé : ils sont conçus pour **identifier des régions spécifiques**, comme les **globules blancs** dans une image.

#### 1. Transformation des Images en Masques Binaires
Un **masque binaire** est une image où chaque pixel est soit un **1** (l'objet d'intérêt, comme les globules blancs) soit un **0** (l'arrière-plan).  
- **0** représente l'arrière-plan,  
- **1** représente l'objet d'intérêt.

Les masques binaires ne contiennent que deux valeurs, contrairement aux images classiques où chaque pixel contient des informations sur les intensités lumineuses.

#### 2. Utilisation des Masques dans la Segmentation
En **deep learning**, ces **masques binaires** sont utilisés pour entraîner des modèles de **segmentation d'images** (comme des réseaux de neurones convolutifs - **CNN**). Le but est que le modèle apprenne à prédire des masques similaires à partir de nouvelles images. Pendant l'entraînement, le modèle utilise les masques comme référence pour ajuster ses prédictions.  
Les masques servent de **guide** pour l'apprentissage supervisé, et le modèle ajuste ses paramètres afin de prédire au mieux les régions contenant l'objet d'intérêt.

---

### Conclusion

En résumé, les images sont transformées en **matrices numériques** qui permettent de les traiter et les analyser. Dans le cadre de la segmentation d'images, ces représentations sont utilisées pour générer des **masques binaires**. Ces masques sont cruciaux pour entraîner des modèles de deep learning à identifier et segmenter des objets d'intérêt dans de nouvelles images. 🔍✨


In [1]:
import os
import shutil

import json
import numpy as np
from PIL import Image
import labelme

# Paths
annotation_path = ".\IMG"  # Chemin des fichiers d'annotations JSON
output_mask_path = ".\Masks"  # Dossier où les masques générés seront enregistrés

# Supprimer et recréer le dossier des masques
if os.path.isdir(output_mask_path):
    shutil.rmtree(output_mask_path)
os.makedirs(output_mask_path, exist_ok=True)

# Fonction pour traiter une annotation
def process_annotation_file(json_path):
    """
    Cette fonction prend en charge le traitement d'un fichier d'annotation JSON généré par LabelMe.
    Elle effectue les étapes suivantes :
    
    1. Chargement du fichier JSON contenant les annotations.
    2. Extraction des dimensions de l'image (hauteur et largeur) depuis le JSON.
    3. Récupération des annotations sous forme de formes (shapes).
    4. Création d'un dictionnaire associant chaque label à une valeur binaire (1 pour l'objet annoté).
    5. Conversion des annotations en une matrice de labels (hauteur x largeur).
    6. Création d'un masque binaire où les objets sont définis par des pixels de valeur 1 (et le reste 0).
    7. Redimensionnement du masque à une taille de 256x256 pixels.
    8. Ajout d'une dimension supplémentaire pour obtenir un format de masque compatible avec les réseaux neuronaux.
    9. Transformation du masque binaire en une image en niveaux de gris (0 ou 255).
    10. Sauvegarde de l'image de masque dans le dossier spécifié, avec le même nom que le fichier JSON mais en format JPG.
    
    Si une erreur se produit à n'importe quelle étape, elle sera capturée et un message d'erreur sera affiché avec les détails de l'erreur.
    
    Arguments:
    - json_path : str : Le chemin du fichier JSON à traiter (chemin complet, y compris le nom du fichier).
    """

    try:
        # Charger le JSON annotation
        with open(json_path) as f:
            data = json.load(f)

        # Obtenir la hauteur et la largeur de l'image à partir du JSON
        height = data["imageHeight"]
        width = data["imageWidth"]

        # Extraire les données des annotations de type polygone que nous avons créées avec labelme
        shapes = data["shapes"]  # Liste de toutes les informations d'annotations (label, points, type de forme, etc.)
        
        # Créer un dictionnaire pour associer les noms des labels aux valeurs (binaire : 1 pour l'objet)
        label_name_to_value = {shape['label']: 1 for shape in shapes}  # Nous avons qu'un seul label >> {'globule_blanc': 1}

        # Convertir les annotations de formes (shapes) en une matrice (numpy array) de labels de dimension (height, width),
        # où chaque pixel est assigné à un label basé sur `label_name_to_value` (par exemple, 1 pour "globule_blanc").
        lbl, _ = labelme.utils.shapes_to_label((height, width), shapes, label_name_to_value)

        # Créer un masque binaire en transformant les valeurs égales à 1 (globule_blanc) en 1,
        # et l'arrière-plan (valeurs égales à 0) reste à 0, car il se peut qu'il y ait plusieurs labels.
        # Puis convertit l'array en entiers sur 8 bits (np.uint8), adapté pour les images ou masques binaires.
        binary_mask = (lbl > 0).astype(np.uint8)

        # Redimensionner le masque à (256, 256) si nécessaire
        resized_mask = np.array(Image.fromarray(binary_mask).resize((256, 256), Image.NEAREST))

        # Ajouter une dimension de canal pour garantir la forme (256, 256, 1),
        # nécessaire pour la compatibilité avec des réseaux neuronaux où TensorFlow s'attend à une image avec un seul canal (grayscale).
        # Lorsque tu ajoutes une dimension à une image en niveaux de gris (qui est une matrice 2D), tu ajoutes une liste supplémentaire autour des valeurs de chaque pixel.
        final_mask = np.expand_dims(resized_mask, axis=-1)

        # Transformer le masque binaire en image en niveaux de gris (valeur encodée allant de 0 à 255) avec un seul canal,
        # compatible avec les modèles de segmentation (comme TensorFlow).
        output_mask = Image.fromarray((final_mask[:, :, 0] * 255).astype(np.uint8), mode="L")

        # Sauvegarder le masque au format JPG dans le dossier spécifié en reprenant le nom de l'image associé
        output_image_path = os.path.join(output_mask_path, os.path.basename(json_path).replace(".json", ".jpg"))
        output_mask.save(output_image_path)

        # Afficher la forme du masque pour vérification, et le chemin d'enregistrement des Masks
        print('------------------------------------')
        print(f"Fichier annotation traité pour : {os.path.basename(json_path)}: {final_mask.shape}")
        print(f"Masque enregistré pour {os.path.basename(json_path)} sous : {output_image_path}")

    except Exception as e:
        # Gestion des erreurs : afficher un message d'erreur si un problème survient pendant le traitement
        print(f"Erreur lors du traitement de {os.path.basename(json_path)} : {e}")


# Liste des fichiers JSON à traiter
json_files = [os.path.join(annotation_path, f) for f in os.listdir(annotation_path) if f.endswith(".json")]

# Traitement de chaque fichier JSON
for json_file in json_files:
    process_annotation_file(json_file)

# Message de fin du processus
print(f"Fin de la génération des Masques. Les masques ont été enregistrés sous : {output_mask_path}")


<>:10: SyntaxWarning: invalid escape sequence '\I'
<>:11: SyntaxWarning: invalid escape sequence '\M'
<>:10: SyntaxWarning: invalid escape sequence '\I'
<>:11: SyntaxWarning: invalid escape sequence '\M'
C:\Users\gbodo\AppData\Local\Temp\ipykernel_25620\3204663407.py:10: SyntaxWarning: invalid escape sequence '\I'
  annotation_path = ".\IMG"  # Chemin des fichiers d'annotations JSON
C:\Users\gbodo\AppData\Local\Temp\ipykernel_25620\3204663407.py:11: SyntaxWarning: invalid escape sequence '\M'
  output_mask_path = ".\Masks"  # Dossier où les masques générés seront enregistrés


------------------------------------
Fichier annotation traité pour : 20170728_201358.json: (256, 256, 1)
Masque enregistré pour 20170728_201358.json sous : .\Masks\20170728_201358.jpg
------------------------------------
Fichier annotation traité pour : 20170728_201531.json: (256, 256, 1)
Masque enregistré pour 20170728_201531.json sous : .\Masks\20170728_201531.jpg
------------------------------------
Fichier annotation traité pour : 20170728_201846.json: (256, 256, 1)
Masque enregistré pour 20170728_201846.json sous : .\Masks\20170728_201846.jpg
------------------------------------
Fichier annotation traité pour : 20170728_202317.json: (256, 256, 1)
Masque enregistré pour 20170728_202317.json sous : .\Masks\20170728_202317.jpg
------------------------------------
Fichier annotation traité pour : 20170728_202444.json: (256, 256, 1)
Masque enregistré pour 20170728_202444.json sous : .\Masks\20170728_202444.jpg
------------------------------------
Fichier annotation traité pour : 20170